In [1]:
library(tidyverse)

# Custom package
library(rutils)

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.0.6     v dplyr   1.0.4
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



In [2]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
matrisome_path <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

In [3]:
matrisome_df <- load_matrisome_df(matrisome_path)


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  Division = col_character(),
  Category = col_character(),
  `Gene Symbol` = col_character(),
  `Gene Name` = col_character(),
  Synonyms = col_character(),
  HGNC_IDs = col_double(),
  `HGNC_IDs Links` = col_double(),
  UniProt_IDs = col_character(),
  Refseq_IDs = col_character(),
  Orthology = col_character(),
  Notes = col_character()
)




In [4]:
hub_con_thresh <- 0.25

In [5]:
for (dset_idx in 1:3) {
    intra_overlap_ls <- read_lines(paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_full_overlap_hub_thresh_", hub_con_thresh, "_gene_list.txt"))
    intra_overlap_df <- tibble(geneID = intra_overlap_ls)
    intra_summary_df <- intra_overlap_df %>%
        inner_join(matrisome_df, by = c("geneID" = "gene_symbol")) %>%
        select(geneID, hgnc_ids, division, category, gene_name) %>%
        rename(gene_id = geneID)
    write_csv(intra_summary_df, paste0(dirs$analysis_dir, "/one_off/", unified_dsets[dset_idx], "_hub_thresh_", hub_con_thresh, "_full_overlap_gene_summary.csv"))
}

In [6]:
inter_summary_df <- read_lines(paste0(dirs$analysis_dir, "/gene_lists/all_cancer_figo_network_hub_thresh_", hub_con_thresh, "_overlap_gene_list.txt")) %>%
    as_tibble() %>%
    rename(geneID = value) %>%
    inner_join(matrisome_df, by = c("geneID" = "gene_symbol")) %>%
    select(geneID, hgnc_ids, division, category, gene_name) %>%
    rename(gene_id = geneID)
write_csv(inter_summary_df, paste0(dirs$analysis_dir, "/one_off/all_cancer_hub_thresh_", hub_con_thresh, "_full_overlap_gene_summary.csv"))